Подключаем необходимые библиотеки.

In [1]:
import os
from pyspark.sql import SparkSession, DataFrame
from pyspark import SparkConf
from pyspark.sql.functions import (
    col,
    lit,
    to_date
)

Сформируем объект конфигурации для Apache Spark, указав необходимые параметры.

In [2]:
def create_spark_configuration() -> SparkConf:
    """
    Создаёт и конфигурирует экземпляр SparkConf для приложения Spark.
    Адаптация для локального выполнения в docker (local[*]) с доступом к HDFS.
    """
    user_name = "jovyan"
    
    conf = SparkConf()
    conf.setAppName("Lab1_Flight_Delay_EDA")
    conf.setMaster("local[*]")
    
    conf.set("spark.sql.adaptive.enabled", "true")

    conf.set("spark.executor.memory", "6g")
    conf.set("spark.executor.cores", "2")
    conf.set("spark.executor.instances", "1")
    conf.set("spark.driver.memory", "4g")
    conf.set("spark.driver.cores", "1")

    conf.set("spark.hadoop.fs.defaultFS", "hdfs://hadoop-namenode:9820")
    conf.set("spark.hadoop.dfs.client.use.datanode.hostname", "true")

    conf.set("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.6.0")
    conf.set("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    conf.set("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkCatalog")
    conf.set("spark.sql.catalog.spark_catalog.type", "hadoop")
    conf.set("spark.sql.catalog.spark_catalog.warehouse", f"hdfs:///user/{user_name}/warehouse")
    conf.set("spark.sql.catalog.spark_catalog.io-impl", "org.apache.iceberg.hadoop.HadoopFileIO")

    return conf


Создаём сам объект конфигурации.

In [3]:
conf = create_spark_configuration()

Создаём и выводим на экран сессию Apache Spark. В процессе создания сессии происходит подключение к кластеру Apache Hadoop, что может занять некоторое время.

In [4]:
spark = SparkSession.builder.config(conf=conf).getOrCreate()
spark

В качестве исходных данных используется датасет Flight Delay Dataset, содержащий информацию о коммерческих авиарейсах в США.
Датасет доступен по ссылке https://www.kaggle.com/datasets/robikscube/flight-delay-dataset-20182022?utm_source=chatgpt.com&select=Combined_Flights_2019.csv

Датасет уже загружен в HDFS по адресу: hdfs://hadoop-namenode:9820/data/flight/Combined_Flights_2019.csv

Указываем путь в HDFS для файла с данными.

In [5]:
path = "hdfs://hadoop-namenode:9820/data/flight/Combined_Flights_2019.csv"

Заполняем датафрейм данными из файла.

In [6]:
df = (spark.read.format("csv")
      .option("header", "true")
      .load(path)
)

Выводим фрагмент датафрейма на экран.

In [7]:
df.limit(10).toPandas().style

,FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,ArrTime,ArrDelayMinutes,AirTime,CRSElapsedTime,ActualElapsedTime,Distance,Year,Quarter,Month,DayofMonth,DayOfWeek,Marketing_Airline_Network,Operated_or_Branded_Code_Share_Partners,DOT_ID_Marketing_Airline,IATA_Code_Marketing_Airline,Flight_Number_Marketing_Airline,Operating_Airline,DOT_ID_Operating_Airline,IATA_Code_Operating_Airline,Tail_Number,Flight_Number_Operating_Airline,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,DestCityName,DestState,DestStateFips,DestStateName,DestWac,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,DistanceGroup,DivAirportLandings
0,2019-04-01,Envoy Air,LIT,ORD,False,False,1212,1209.0,0.0,-3.0,1350.0,0.0,83.0,113.0,101.0,552.0,2019,2,4,1,1,AA,AA_CODESHARE,19805,AA,3315,MQ,20398,MQ,N253NN,3315,12992,1299206,32600,"Little Rock, AR",AR,5,Arkansas,71,13930,1393007,30977,"Chicago, IL",IL,17,Illinois,41,0.0,-1.0,1200-1259,10.0,1219.0,1342.0,8.0,1405,-15.0,0.0,-1.0,1400-1459,3,0
1,2019-04-02,Envoy Air,LIT,ORD,False,False,1212,1200.0,0.0,-12.0,1348.0,0.0,89.0,113.0,108.0,552.0,2019,2,4,2,2,AA,AA_CODESHARE,19805,AA,3315,MQ,20398,MQ,N242NN,3315,12992,1299206,32600,"Little Rock, AR",AR,5,Arkansas,71,13930,1393007,30977,"Chicago, IL",IL,17,Illinois,41,0.0,-1.0,1200-1259,10.0,1210.0,1339.0,9.0,1405,-17.0,0.0,-2.0,1400-1459,3,0
2,2019-04-03,Envoy Air,LIT,ORD,False,False,1212,1203.0,0.0,-9.0,1342.0,0.0,82.0,113.0,99.0,552.0,2019,2,4,3,3,AA,AA_CODESHARE,19805,AA,3315,MQ,20398,MQ,N247NN,3315,12992,1299206,32600,"Little Rock, AR",AR,5,Arkansas,71,13930,1393007,30977,"Chicago, IL",IL,17,Illinois,41,0.0,-1.0,1200-1259,11.0,1214.0,1336.0,6.0,1405,-23.0,0.0,-2.0,1400-1459,3,0
3,2019-04-04,Envoy Air,LIT,ORD,False,False,1212,1435.0,143.0,143.0,1621.0,136.0,83.0,113.0,106.0,552.0,2019,2,4,4,4,AA,AA_CODESHARE,19805,AA,3315,MQ,20398,MQ,N220NN,3315,12992,1299206,32600,"Little Rock, AR",AR,5,Arkansas,71,13930,1393007,30977,"Chicago, IL",IL,17,Illinois,41,1.0,9.0,1200-1259,17.0,1452.0,1615.0,6.0,1405,136.0,1.0,9.0,1400-1459,3,0
4,2019-04-05,Envoy Air,LIT,ORD,False,False,1212,1216.0,4.0,4.0,1410.0,5.0,83.0,113.0,114.0,552.0,2019,2,4,5,5,AA,AA_CODESHARE,19805,AA,3315,MQ,20398,MQ,N255NN,3315,12992,1299206,32600,"Little Rock, AR",AR,5,Arkansas,71,13930,1393007,30977,"Chicago, IL",IL,17,Illinois,41,0.0,0.0,1200-1259,18.0,1234.0,1357.0,13.0,1405,5.0,0.0,0.0,1400-1459,3,0
5,2019-04-06,Envoy Air,LIT,ORD,False,False,1212,1305.0,53.0,53.0,1501.0,56.0,90.0,113.0,116.0,552.0,2019,2,4,6,6,AA,AA_CODESHARE,19805,AA,3315,MQ,20398,MQ,N269NN,3315,12992,1299206,32600,"Little Rock, AR",AR,5,Arkansas,71,13930,1393007,30977,"Chicago, IL",IL,17,Illinois,41,1.0,3.0,1200-1259,13.0,1318.0,1448.0,13.0,1405,56.0,1.0,3.0,1400-1459,3,0
6,2019-04-07,Envoy Air,LIT,ORD,False,False,1212,1219.0,7.0,7.0,1401.0,0.0,81.0,113.0,102.0,552.0,2019,2,4,7,7,AA,AA_CODESHARE,19805,AA,3315,MQ,20398,MQ,N252NN,3315,12992,1299206,32600,"Little Rock, AR",AR,5,Arkansas,71,13930,1393007,30977,"Chicago, IL",IL,17,Illinois,41,0.0,0.0,1200-1259,11.0,1230.0,1351.0,10.0,1405,-4.0,0.0,-1.0,1400-1459,3,0
7,2019-04-08,Envoy Air,LIT,ORD,False,False,1212,1222.0,10.0,10.0,1416.0,11.0,90.0,113.0,114.0,552.0,2019,2,4,8,1,AA,AA_CODESHARE,19805,AA,3315,MQ,20398,MQ,N266NN,3315,12992,1299206,32600,"Little Rock, AR",AR,5,Arkansas,71,13930,1393007,30977,"Chicago, IL",IL,17,Illinois,41,0.0,0.0,1200-1259,12.0,1234.0,1404.0,12.0,1405,11.0,0.0,0.0,1400-1459,3,0
8,2019-04-09,Envoy Air,LIT,ORD,False,False,1212,1201.0,0.0,-11.0,1355.0,0.0,86.0,113.0,114.0,552.0,2019,2,4,9,2,AA,AA_CODESHARE,19805,AA,3315,MQ,20398,MQ,N242NN,3315,12992,1299206,32600,"Little Rock, AR",AR,5,Arkansas,71,13930,1393007,30977,"Chicago, IL",IL,17,Illinois,41,0.0,-1.0,1200-1259,13.0,1214.0,1340.0,15.0,1405,-10.0,0.0,-1.0,1400-1459,3

Очевидно, что в целях сохранения ясности изложения и сокращения расчетного времени имеет смысл рассматривать не все солбцы датасета. Оставим следующие колонки, удалив остальные:

| Название столбца | Описание                         |
| ---------------- | -------------------------------- |
| FlightDate       | Дата выполнения рейса            |
| Airline          | Название авиакомпании            |
| Origin           | Аэропорт вылета                  |
| Dest             | Аэропорт назначения              |
| Cancelled        | Факт отмены рейса                |
| Diverted         | Факт отклонения рейса            |
| DepDelayMinutes  | Задержка вылета (минуты)         |
| ArrDelayMinutes  | Задержка прибытия (минуты)       |
| AirTime          | Время в полёте                   |
| Distance         | Дистанция перелёта               |
| Year             | Год                              |
| Month            | Месяц                            |
| DayOfWeek        | День недели                      |
| DepDel15         | Задержка вылета более 15 минут   |
| ArrDel15         | Задержка прибытия более 15 минут |


In [29]:
df = df.select(
    "FlightDate",
    "Airline",
    "Origin",
    "Dest",
    "Cancelled",
    "Diverted",
    "DepDelayMinutes",
    "ArrDelayMinutes",
    "AirTime",
    "Distance",
    "Year",
    "Month",
    "DayOfWeek",
    "DepDel15",
    "ArrDel15"
)

df.show()


+----------+---------+------+----+---------+--------+---------------+---------------+-------+--------+----+-----+---------+--------+--------+
|FlightDate|  Airline|Origin|Dest|Cancelled|Diverted|DepDelayMinutes|ArrDelayMinutes|AirTime|Distance|Year|Month|DayOfWeek|DepDel15|ArrDel15|
+----------+---------+------+----+---------+--------+---------------+---------------+-------+--------+----+-----+---------+--------+--------+
|2019-04-01|Envoy Air|   LIT| ORD|    False|   False|            0.0|            0.0|   83.0|   552.0|2019|    4|        1|     0.0|     0.0|
|2019-04-02|Envoy Air|   LIT| ORD|    False|   False|            0.0|            0.0|   89.0|   552.0|2019|    4|        2|     0.0|     0.0|
|2019-04-03|Envoy Air|   LIT| ORD|    False|   False|            0.0|            0.0|   82.0|   552.0|2019|    4|        3|     0.0|     0.0|
|2019-04-04|Envoy Air|   LIT| ORD|    False|   False|          143.0|          136.0|   83.0|   552.0|2019|    4|        4|     1.0|     1.0|
|2019-

Выведем на экран метаданные датасета.

In [30]:
df.printSchema()

root
 |-- FlightDate: string (nullable = true)
 |-- Airline: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Cancelled: string (nullable = true)
 |-- Diverted: string (nullable = true)
 |-- DepDelayMinutes: string (nullable = true)
 |-- ArrDelayMinutes: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- DepDel15: string (nullable = true)
 |-- ArrDel15: string (nullable = true)



Видно, что все поля загружены в строковом формате, что требует явного приведения типов данных.

In [31]:
def transform_dataframe(data: DataFrame) -> DataFrame:
    """
    Приводит столбцы датафрейма к корректным типам данных.

    Args:
        data (DataFrame): Исходный DataFrame.

    Returns:
        DataFrame: Преобразованный DataFrame.
    """
    data = data.withColumn("FlightDate", col("FlightDate").cast("date"))

    data = data.withColumn("Cancelled", col("Cancelled").cast("Boolean"))
    data = data.withColumn("Diverted", col("Diverted").cast("Boolean"))

    data = data.withColumn(
        "DepDelayMinutes", col("DepDelayMinutes").cast("Float")
    )
    data = data.withColumn(
        "ArrDelayMinutes", col("ArrDelayMinutes").cast("Float")
    )
    data = data.withColumn("AirTime", col("AirTime").cast("Float"))
    data = data.withColumn("Distance", col("Distance").cast("Float"))

    data = data.withColumn("Year", col("Year").cast("Integer"))
    data = data.withColumn("Month", col("Month").cast("Integer"))
    data = data.withColumn("DayOfWeek", col("DayOfWeek").cast("Integer"))

    data = data.withColumn("DepDel15", col("DepDel15").cast("float").cast("Boolean"))
    data = data.withColumn("ArrDel15", col("ArrDel15").cast("float").cast("Boolean"))

    return data

In [32]:
df = transform_dataframe(df)

In [33]:
df.show()

+----------+---------+------+----+---------+--------+---------------+---------------+-------+--------+----+-----+---------+--------+--------+
|FlightDate|  Airline|Origin|Dest|Cancelled|Diverted|DepDelayMinutes|ArrDelayMinutes|AirTime|Distance|Year|Month|DayOfWeek|DepDel15|ArrDel15|
+----------+---------+------+----+---------+--------+---------------+---------------+-------+--------+----+-----+---------+--------+--------+
|2019-04-01|Envoy Air|   LIT| ORD|    false|   false|            0.0|            0.0|   83.0|   552.0|2019|    4|        1|   false|   false|
|2019-04-02|Envoy Air|   LIT| ORD|    false|   false|            0.0|            0.0|   89.0|   552.0|2019|    4|        2|   false|   false|
|2019-04-03|Envoy Air|   LIT| ORD|    false|   false|            0.0|            0.0|   82.0|   552.0|2019|    4|        3|   false|   false|
|2019-04-04|Envoy Air|   LIT| ORD|    false|   false|          143.0|          136.0|   83.0|   552.0|2019|    4|        4|    true|    true|
|2019-

In [34]:
df.printSchema()

root
 |-- FlightDate: date (nullable = true)
 |-- Airline: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Cancelled: boolean (nullable = true)
 |-- Diverted: boolean (nullable = true)
 |-- DepDelayMinutes: float (nullable = true)
 |-- ArrDelayMinutes: float (nullable = true)
 |-- AirTime: float (nullable = true)
 |-- Distance: float (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepDel15: boolean (nullable = true)
 |-- ArrDel15: boolean (nullable = true)



Видно, что теперь столбцы датафрейма содержат значения корректных типов.

Полученный датафрейм сохраним для дальнейшего использования. Сохранение выполним в таблицу `Apache Iceberg`.

`Apache Iceberg` — это поддерживающий высокую производительность табличный формат для больших данных.

Сначала создадим базу данных, в которой будет расположена таблица.

Создадим инструкцию SQL для добавления базы данных в каталог Apache Spark.

In [36]:
database_name = "gordeev_database"

In [37]:
create_database_sql = f"""
CREATE DATABASE IF NOT EXISTS spark_catalog.{database_name}
"""

In [38]:
spark.sql(create_database_sql)

DataFrame[]

И, наконец, записываем преобразованный датафрейм в таблицу sobd_lab1_flight_table.

In [39]:
# Сохранение DataFrame в виде таблицы
df.writeTo(f"spark_catalog.{database_name}.sobd_lab1_flight_table").using("iceberg").create()

После успешной записи можно посмотреть, какие таблицы входят в базу данных.

In [41]:
for table in spark.catalog.listTables("gordeev_database"):
    print(table.name)

sobd_lab1_table
sobd_lab1_flight_table


In [42]:
df_table = spark.table("gordeev_database.sobd_lab1_flight_table")
df_table.show(10)

+----------+---------+------+----+---------+--------+---------------+---------------+-------+--------+----+-----+---------+--------+--------+
|FlightDate|  Airline|Origin|Dest|Cancelled|Diverted|DepDelayMinutes|ArrDelayMinutes|AirTime|Distance|Year|Month|DayOfWeek|DepDel15|ArrDel15|
+----------+---------+------+----+---------+--------+---------------+---------------+-------+--------+----+-----+---------+--------+--------+
|2019-04-01|Envoy Air|   LIT| ORD|    false|   false|            0.0|            0.0|   83.0|   552.0|2019|    4|        1|   false|   false|
|2019-04-02|Envoy Air|   LIT| ORD|    false|   false|            0.0|            0.0|   89.0|   552.0|2019|    4|        2|   false|   false|
|2019-04-03|Envoy Air|   LIT| ORD|    false|   false|            0.0|            0.0|   82.0|   552.0|2019|    4|        3|   false|   false|
|2019-04-04|Envoy Air|   LIT| ORD|    false|   false|          143.0|          136.0|   83.0|   552.0|2019|    4|        4|    true|    true|
|2019-

In [43]:
spark.stop()